In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        

In [3]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [4]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [5]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [6]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [7]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [8]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [9]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [10]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
         #   particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
          #  particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [11]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [12]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [13]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [14]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)   
    set_random_direction(particle)

In [15]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    

In [16]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        process_scattering(particle, c_s)
        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [17]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [18]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    
    while not particle.is_terminated():
        set_random_direction(particle)
        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [19]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [20]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       

In [21]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [22]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [23]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [24]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [25]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [26]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [27]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [28]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [37]:
cs_fission_pu_239 = [0.08544, 0.0936]

cs_capture_pu_239 = [0.0144, 0.00480]

cs_scattering_pu_239 = [[0.23616, 0.0], [0.0432, 0.0792]]

cs_total_pu_239 = [0.3360, 0.2208]

cs_production_neutrons_pu_239 = [2.93, 3.10]

cs_xi_table_pu_239 = [ 0.425, 0.575]


pu_239 = Sig(2, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [68]:
cs_fission_pu_239 = [0.9, 0.06, 0.006]

cs_capture_pu_239 = [0.2, 0.04, 0.006]

cs_scattering_pu_239 = [[2.0, 0.0, 0.0], [0.275, 0.6, 0.0], [0.033, 0.171, 0.024]]

cs_total_pu_239 = [3.10, 0.975, 0.24]

cs_production_neutrons_pu_239 = [2., 2.5, 3.]

cs_xi_table_pu_239 = [0.0, 0.04, 0.96]


pu_239 = Sig(3, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [ ]:
cs_fission_pu_239 = [0.9, 0.06, 0.006]

cs_capture_pu_239 = [0.2, 0.04, 0.006]

cs_scattering_pu_239 = [[2.0, 0.0, 0.0], [0.275, 0.6, 0.0], [0.033, 0.171, 0.024]]

cs_total_pu_239 = []

cs_production_neutrons_pu_239 = [3.0, 2.5, 2.0, 2.0, 2.5, 3.0]

cs_xi_table_pu_239 = [0.48, 0.02, 0.0, 0.0, 0.02, 0.48]


pu_239 = Sig(6, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [69]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000, -1000, 1000, 1000, 1000 )
slab.set_zero_point(0., 0., 0.)

In [70]:
cells = [slab]

In [71]:
estimators = []

In [72]:
materials = [pu_239]

In [73]:
U235_Reactor = Universe(cells, materials)

In [74]:
test_number_of_particles = 10
test_number_interations = 1000
test_number_inactive = 200

In [76]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)

 num1  7
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.5843464405061434   1.600023047361225  0.0033326359861474745
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.5987711264688165   1.6000018283630486  0.003275732376717254
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.586252563001261   1.5997726739403524  0.0032288161297683964
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.6067566471929862   1.5998871653051494  0.003177506928242316
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.6287545722887564   1.6003527686435948  0.003160323017906252
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.640165831126176   1.6009847220163342  0.0031733166649795925
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.5415027368673067   1.6000553159983806  0.003258688174624579
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.5954512390669924   1.59998448404559  0.0032089446594562874
i == 266
 keff_cycle , k_effective_exp, std_k_effective 1.60722283

 keff_cycle , k_effective_exp, std_k_effective 1.5938020473257397   1.5987576405915591  0.0023222532767295224
i == 329
 keff_cycle , k_effective_exp, std_k_effective 1.6284150013941445   1.5989875426132845  0.0023156219432272133
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.6292765460657637   1.5992205349475344  0.0023095230003064445
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.6189294952124134   1.5993709850258924  0.0022967581894574817
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.5795522241899411   1.5992208428983472  0.0022842318422811297
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.626397139998156   1.599425175959248  0.002276182160488352
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.5690679876022946   1.5991986297774796  0.00227046250001283
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.6145116913991902   1.5993120598635664  0.002256434350597785
i == 336
 keff_cycle , k_effective_exp, std_k_effective 1.63789

 keff_cycle , k_effective_exp, std_k_effective 1.599589696434419   1.599326893067094  0.0018298612769431977
i == 399
 keff_cycle , k_effective_exp, std_k_effective 1.6241742606778964   1.5994517542108668  0.001824919289785896
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.559395401967411   1.5992514724496496  0.0018267840304662802
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.6429865410967914   1.5994690598558543  0.0018306498303133343
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.5511142142232297   1.5992296794319303  0.0018372263906101113
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.6054173317364946   1.5992601604777654  0.0018284077018301717
i == 404
 keff_cycle , k_effective_exp, std_k_effective 1.628083264492325   1.599401450203327  0.0018249006187373328
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.557991784995111   1.5991994518364576  0.0018271768510171435
i == 406
 keff_cycle , k_effective_exp, std_k_effective 1.555656

 keff_cycle , k_effective_exp, std_k_effective 1.6150749841229974   1.5988272804731267  0.0015552171144958696
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.5739312947954731   1.5987347303404962  0.0015521864896368544
i == 470
 keff_cycle , k_effective_exp, std_k_effective 1.573954683730347   1.598642952390088  0.001549148008036209
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.5533296688159859   1.598475744701623  0.0015524518740650684
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.625873984248115   1.5985764735234849  0.0015500102396938955
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.6072151076905887   1.5986081168720825  0.001544646263555207
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.6473689394934004   1.5987860760787296  0.001549253367309991
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.6416288270015424   1.5989418679002672  0.0015514513171948736
i == 476
 keff_cycle , k_effective_exp, std_k_effective 1.578827

...

 keff_cycle , k_effective_exp, std_k_effective 1.6340340339525796   1.5996255965058512  0.0011391497993128998
i == 679
 keff_cycle , k_effective_exp, std_k_effective 1.6521919655303943   1.599735338403606  0.00114205399843711
i == 680
 keff_cycle , k_effective_exp, std_k_effective 1.6233686090988964   1.599784574384221  0.00114073528330169
i == 681
 keff_cycle , k_effective_exp, std_k_effective 1.5981930599823229   1.5997812656224708  0.001138366030480898
i == 682
 keff_cycle , k_effective_exp, std_k_effective 1.617107261663267   1.599817211672348  0.001136570391113933
i == 683
 keff_cycle , k_effective_exp, std_k_effective 1.6080940361121898   1.599834347954832  0.0011343442466403163
i == 684
 keff_cycle , k_effective_exp, std_k_effective 1.6011583581885287   1.5998370835131661  0.0011320014393373027
i == 685
 keff_cycle , k_effective_exp, std_k_effective 1.6002223991785187   1.5998378779784554  0.0011296652839308766
i == 686
 keff_cycle , k_effective_exp, std_k_effective 1.5740366892

 keff_cycle , k_effective_exp, std_k_effective 1.5975744807953693   1.599551778090347  0.0010398285267580229
i == 749
 keff_cycle , k_effective_exp, std_k_effective 1.6029829722308233   1.5995580279885993  0.0010379515742535178
i == 750
 keff_cycle , k_effective_exp, std_k_effective 1.6069158170144642   1.5995714057868282  0.001036149035303718
i == 751
 keff_cycle , k_effective_exp, std_k_effective 1.611531411920958   1.5995931117870716  0.0010344945826823773
i == 752
 keff_cycle , k_effective_exp, std_k_effective 1.6249919988566335   1.599639124263647  0.001033643424641268
i == 753
 keff_cycle , k_effective_exp, std_k_effective 1.6137121216301233   1.5996645727218142  0.001032086368054334
i == 754
 keff_cycle , k_effective_exp, std_k_effective 1.625947267471463   1.599712014409088  0.0010313134776833578
i == 755
 keff_cycle , k_effective_exp, std_k_effective 1.5861008465294284   1.5996874897822777  0.0010297456612146688
i == 756
 keff_cycle , k_effective_exp, std_k_effective 1.6343930

 keff_cycle , k_effective_exp, std_k_effective 1.5776555799073657   1.5996999023494742  0.0009776037049760093
i == 819
 keff_cycle , k_effective_exp, std_k_effective 1.5781269119661918   1.5996650509918275  0.0009766451291729494
i == 820
 keff_cycle , k_effective_exp, std_k_effective 1.6366733491859657   1.5997247417953664  0.000976893960971526
i == 821
 keff_cycle , k_effective_exp, std_k_effective 1.6094211565573675   1.59974035598983  0.0009754445724155669
i == 822
 keff_cycle , k_effective_exp, std_k_effective 1.5419897097310176   1.5996475092916649  0.0009782909398752596
i == 823
 keff_cycle , k_effective_exp, std_k_effective 1.579125862900689   1.5996145692493038  0.0009772746838147667
i == 824
 keff_cycle , k_effective_exp, std_k_effective 1.5869721934112473   1.5995943090316145  0.0009759176070182835
i == 825
 keff_cycle , k_effective_exp, std_k_effective 1.6064626169574951   1.599605298324296  0.0009744168572432066
i == 826
 keff_cycle , k_effective_exp, std_k_effective 1.6106

 keff_cycle , k_effective_exp, std_k_effective 1.5977770740848696   1.5994302810527228  0.0009246051069979972
i == 889
 keff_cycle , k_effective_exp, std_k_effective 1.6008107896280706   1.5994322846936158  0.0009232643536052393
i == 890
 keff_cycle , k_effective_exp, std_k_effective 1.6075744821686775   1.5994440849798115  0.0009220008347535289
i == 891
 keff_cycle , k_effective_exp, std_k_effective 1.6002604022102653   1.599445266336151  0.0009206663264738953
i == 892
 keff_cycle , k_effective_exp, std_k_effective 1.55835395627585   1.5993858858302834  0.0009212506399737116
i == 893
 keff_cycle , k_effective_exp, std_k_effective 1.5861625562188366   1.5993668045465728  0.0009201181872337114
i == 894
 keff_cycle , k_effective_exp, std_k_effective 1.590168475692291   1.599353550470414  0.0009188870056706503
i == 895
 keff_cycle , k_effective_exp, std_k_effective 1.5973008571652838   1.599350596954867  0.0009175686670170216
i == 896
 keff_cycle , k_effective_exp, std_k_effective 1.59825

 keff_cycle , k_effective_exp, std_k_effective 1.5656015520172901   1.5994229251827017  0.0008799511299415954
i == 959
 keff_cycle , k_effective_exp, std_k_effective 1.5616947328763893   1.5993732174194522  0.0008801957167244464
i == 960
 keff_cycle , k_effective_exp, std_k_effective 1.5377951102284133   1.5992921935942008  0.0008827630252130639
i == 961
 keff_cycle , k_effective_exp, std_k_effective 1.591216366321964   1.5992815814690073  0.0008816661265169654
i == 962
 keff_cycle , k_effective_exp, std_k_effective 1.6027127491856965   1.59928608431378  0.0008805198376517396
i == 963
 keff_cycle , k_effective_exp, std_k_effective 1.550438674684395   1.599222064117673  0.0008816924004732871
i == 964
 keff_cycle , k_effective_exp, std_k_effective 1.5855463147061934   1.5992041639221084  0.0008807195217420532
i == 965
 keff_cycle , k_effective_exp, std_k_effective 1.5811562545083222   1.5991805718836591  0.0008798838406280063
i == 966
 keff_cycle , k_effective_exp, std_k_effective 1.6391

In [78]:
difference = (1.600000  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  88.29215034458215  with standart deviation [pcm]+- 85.57398367004944


In [39]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [40]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)